In [1]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from scipy.sparse import *

In [15]:
class Quadmesh(AbstractMesh):
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals    = None #npArray (Nx3)
        self.face_labels     = None #npArray (Nx1)
        self.faces           = None #npArray (Nx4)
        self.__face2face       = None #npArray (Nx3?)
        
        super(Quadmesh, self).__init__()
        
        if vertices and faces:
            self.vertices = np.array(vertices)
            self.faces = np.array(faces)
            
            if(faces_normals):
                self.face_normals = np.array(face_normals)
                
            if(labels):
                self.labels = np.array(labels)
                            
            self.__load_operations()
            
    
    def show(self):
        pass
    
    
    @property
    def num_faces(self):
        
        return self.faces.shape[0]
       
#_______________________________Operations on Mesh Elements__________________________________________________________________________________
        
    def add_face(self, face_id0, face_id1, face_id2, face_id3):
        
        self.add_faces([face_id0, face_id1, face_id2, face_id3])
        
        
    def add_faces(self, new_faces):
        
        new_faces = np.array(new_faces)
        self.faces = np.concatenate([self.faces, new_faces])
        
        if new_faces[(new_faces[:,0] > self.num_vertices) | 
                     (new_faces[:,1] > self.num_vertices) | 
                     (new_faces[:,2] > self.num_vertices) | 
                     (new_faces[:,3] > self.num_vertices)].shape[0] > n_vert:
            raise Exception('The Id of a vertex must be lesser than the number of vertices')

        self.faces = np.concatenate([self.faces, new_faces])
        self.__compute_adjacencies()
        
        
    def remove_face(self, face_id):
        
        self.remove_faces([face_id])
        
        
    def remove_faces(self, face_ids):
        
        face_ids = np.array(face_ids)
        mask = np.ones(self.num_faces)
        mask[face_ids] = 0
        mask = mask.astype(np.bool)
        
        self.faces = self.faces[mask]
        self.__compute_adjacencies()
        
        
    def removeVertex(self, vtx_id):
        
        self.remove_vertices([vtx_id])
                
        
    def removeVertices(self, vtx_ids):
        
        for vtx_id in vtx_ids:
            
            self.vertices = np.delete(self.vertices, vtx_id)
            self.faces = self.faces[(self.faces[:,0] != vtx_id) & 
                                    (self.faces[:,1] != vtx_id) & 
                                    (self.faces[:,2] != vtx_id) &
                                    (self.faces[:,3] != vtx_id)]
            
            self.faces[(self.faces[:,0] > vtx_id)][0] -=1
            self.faces[(self.faces[:,1] > vtx_id)][1] -=1
            self.faces[(self.faces[:,2] > vtx_id)][2] -=1
            self.faces[(self.faces[:,3] > vtx_id)][3] -=1
        
        self.__compute_adjacencies()
        
#____________________________________________________________________________________________________________________________________________________
        
    def __compute_adjacencies(self):
        
        map_ = dict()
        adjs =  [[] for i in range(self.num_faces)]
        vtx2vtx = [[] for i in range(self.num_vertices)]
        vtx2face = [[] for i in range(self.num_vertices)]


        edges = np.c_[self.faces[:,0], self.faces[:,1], 
                      self.faces[:,1], self.faces[:,2], 
                      self.faces[:,2], self.faces[:,3], 
                      self.faces[:,3], self.faces[:,0]]
        edges.shape = (-1, 2)
        faces_idx = np.repeat(np.array(range(self.num_faces)), 4)

        for e, f in zip(edges, faces_idx):
            
            vtx2vtx[e[0]].append(e[1])
            vtx2face[e[0]].append(f)
            vtx2face[e[1]].append(f)
            e = (e[0], e[1])

            try:
                tmp = map_[e]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(e[1], e[0])] = f
            else:
                adjs[f].append(map_[e])
                adjs[map_[e]].append(f)

        self.__face2face = np.array([np.array(a) for a in adjs])
        self._AbstractMesh__vtx2vtx = np.array([np.array(a) for a in vtx2vtx])
        self._AbstractMesh__vtx2face = np.array([np.array(a) for a in vtx2face])

    
    def __load_operations(self):
        
        self.__compute_adjacencies()
        self._AbstractMesh__update_bounding_box()
        #self.__compute_metrics()
        
        
    def load_from_file(self, filename):
        
        self.vertices, self.faces, self.face_normals = utils.read_obj(filename)
        self.__load_operations()
        
        
    def save_file(self, filename):
        raise NotImplementedError
    
    
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroid is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
            
        return self._AbstractMesh__simplex_centroid
    
    
    @property
    def face2face(self):
        
        return self.__face2face
    
    def __comput_metrics(self):
        
        self.simplex_metrics['aspect_ratio'] = aspect_ratio(self.vertices, self.faces)
        
    
    @property
    def export_triangles(self):
        
        tris = np.c_[self.faces[:,:3], self.faces[:,2:], self.faces[:,0]]
        tris.shape = (-1, 3)
        return tris
    
    
    

    
    

In [16]:
qm = Quadmesh()

In [17]:
qm.load_from_file('bunny_quad.obj')

In [18]:
qm.num_vertices

2414

In [19]:
qm.export_triangles

array([[   2,    1,    0],
       [   0,    3,    2],
       [   5,    4,    0],
       ...,
       [2408, 2409, 2411],
       [2413, 2412, 2410],
       [2410, 2411, 2413]])